In [173]:
# imports and options
import pandas as pd
import numpy as np
pd.set_option("display.max_columns", None)

In [174]:
# read timelines and match ids and tournament ids
timelines_df = pd.read_csv('../data/match_timelines.csv')
team_names_df = pd.read_csv('../data/match_ids.csv')
tournaments_df = pd.read_csv('../data/tournament_ids.csv')

# select correct tournaments
tournaments_df = tournaments_df[tournaments_df['tournament_id'].isin([57, 58, 59, 60, 61, 62, 102, 103, 104, 105, 106, 107, 152, 153, 154, 155, 156, 157])]

In [175]:
# add column of age group and gender
tournaments_df['age'] = ['T12', 'P12', 'T13', 'T14', 'P13', 'P14',
                         'P13', 'T13', 'T12', 'P12', 'T14', 'P14',
                         'P13', 'T12', 'T13', 'T14', 'P14', 'P12']

# create year column from name
tournaments_df['year'] = tournaments_df['name'].str.extract(r'(\d{4})')

In [176]:
# list international teams
int_teams = ['Nords', 'jylland', 'Shamrock', 'Hamma', 'ndby', 'Vitesse', 'Bromma', 'Liding']

In [177]:
# create boolean variables for international teams
team_names_df['home_team_int'] = team_names_df['home_team'].apply(lambda x: 1 if any(sub in x for sub in int_teams) else 0)
team_names_df['away_team_int'] = team_names_df['away_team'].apply(lambda x: 1 if any(sub in x for sub in int_teams) else 0)

In [178]:
# add home and away team names to main dataframe
timelines_df = timelines_df.merge(team_names_df, on=['tournament_id', 'match_id'], how='left')

# add year and age group to main dataframe
timelines_df = timelines_df.merge(tournaments_df.drop('name', axis=1), on=['tournament_id'], how='left')

# convert times to datetime objects
# Convert to datetime
timelines_df['start_time'] = pd.to_datetime(timelines_df['start_time'], format='mixed')
timelines_df['end_time'] = pd.to_datetime(timelines_df['end_time'], format='mixed')

In [179]:
# Function to calculate Euclidean distance
def euclidean_distance(x1, y1, x2, y2):
    return np.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)

# Apply distance calculation for "Pass" and "Dribble" events
timelines_df['distance'] = np.where(
    timelines_df['event_type'].isin(['Pass', 'Dribble']), 
    euclidean_distance(timelines_df['start_position_x'], timelines_df['start_position_y'], timelines_df['end_position_x'], timelines_df['end_position_y']),
    np.nan  # Assign NaN for other events
)

# calculate duration of event for possible events
timelines_df['duration'] = timelines_df['end_time'] - timelines_df['start_time']
timelines_df['duration'] = timelines_df['duration'].dt.total_seconds() #convert to seconds

# calculate speed for possible events
timelines_df['speed'] = timelines_df['distance'] / timelines_df['duration']
timelines_df.loc[(timelines_df['speed'].notna()) & (timelines_df['age'] == 'P14')]

# create column of international level (0=f-f, 1=i-f, 2=i-i)
timelines_df['int_level'] = timelines_df['home_team_int'] + timelines_df['away_team_int']

In [181]:
# group by "age" and "team_type", then compute statistics for "speed"
speed_stats = timelines_df.groupby(["age", "int_level"])["speed"].describe()
speed_stats

count      mean       std       min       25%       50%  \
age int_level                                                             
P12 0          6536.0  8.386466  3.187714  0.032197  6.553304  8.337476   
    1          9894.0  8.339230  3.195743  0.048379  6.469891  8.296135   
    2          1701.0  8.610807  3.275831  0.128897  6.705848  8.534817   
P13 0          4857.0  8.826911  3.319490  0.107981  6.811930  8.746309   
    1          8869.0  8.849077  3.365715  0.043011  6.755605  8.750902   
    2          1759.0  8.804571  3.314532  0.060606  6.802764  8.784886   
P14 0          9507.0  8.972046  3.634402  0.077784  6.878772  8.905940   
    1          9032.0  9.153743  3.585598  0.004947  6.956367  9.040405   
    2          1502.0  9.230459  3.762354  0.079672  6.898512  9.188117   
T12 0          3939.0  7.654549  2.724556  0.139159  6.151501  7.666611   
    1          6006.0  7.658974  2.666524  0.047166  6.139767  7.615177   
    2           716.0  7.470151  2.507544  0.268554  5.930213  7.308207   
T13 0          5994.0  8.426568  2.962098  0.019788  6.694379  8.432745   
    1          7499.0  8.405685  2.889415  0.088319  6.669889  8.415075   
    2          1100.0  8.660388  2.923507  0.304864  6.885587  8.493951   
T14 0          7391.0  8.505154  3.069090  0.075351  6.797416  8.512248   
    1          8066.0  8.430695  3.057611  0.028060  6.672907  8.371982   
    2          1088.0  8.648922  2.948858  0.110721  6.777169  8.555175   

                     75%         max  
age int_level                         
P12 0          10.225560   34.983379  
    1          10.287914   57.672094  
    2          10.695031   28.339725  
P13 0          10.850972   29.916053  
    1          10.867723   53.133373  
    2          10.835265   18.838903  
P14 0          11.007321  119.385814  
    1          11.358998   35.054925  
    2          11.598764   22.221114  
T12 0           9.226173   21.992262  
    1           9.233793   22.580080  
    2           9.128832   16.087465  
T13 0          10.236941   29.858282  
    1          10.176143   20.915439  
    2          10.472324   17.585141  
T14 0          10.364226   23.821209  
    1          10.270113   48.319794  
    2          10.500375   19.217695